In [1]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [2]:
path="../input/tabular-playground-series-nov-2022/submission_files"
li1 = os.listdir(path)
li = [path+"/"+a for a in li1]
len(li)

5000

In [3]:
datasets = []
for a in li[:1000]:
    df1 = pd.read_csv(a,index_col="id")
    df1.columns=[a.split("/")[-1].split(".csv")[0]]
    datasets.append(df1)
df=pd.concat(datasets,axis=1)

In [4]:
train=df.iloc[:20000]
test=df.iloc[20000:]
del datasets,df

In [5]:
labels = pd.read_csv("../input/tabular-playground-series-nov-2022/train_labels.csv",index_col="id")

In [6]:
params = {"boosting_type":["gbdt","dart","goss","rf"],
         "num_leaves":[5,10,25,50,100,500],
         "learning_rate":[0.1,0.01,0.05,0.001,0.005,0.2,0.3],
         "n_estimators":[50,100,200,500,1000],
         "min_child_samples":[20,50,100,500,1000,5000,10000]}

In [7]:
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV

In [8]:
model = lgb.LGBMClassifier(device="gpu")

In [9]:
clf = RandomizedSearchCV(model,params,scoring=log_loss,verbose=2,n_iter=10,cv=2)

In [10]:
clf.fit(train,labels.label.values)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END boosting_type=goss, learning_rate=0.1, min_child_samples=5000, n_estimators=50, num_leaves=10; total time=   2.3s
[CV] END boosting_type=goss, learning_rate=0.1, min_child_samples=5000, n_estimators=50, num_leaves=10; total time=   1.3s


[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /usr/local/src/LightGBM/src/boosting/rf.hpp, line 35 .



[CV] END boosting_type=rf, learning_rate=0.2, min_child_samples=100, n_estimators=1000, num_leaves=50; total time=   2.1s


[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /usr/local/src/LightGBM/src/boosting/rf.hpp, line 35 .



[CV] END boosting_type=rf, learning_rate=0.2, min_child_samples=100, n_estimators=1000, num_leaves=50; total time=   2.1s
[CV] END boosting_type=goss, learning_rate=0.1, min_child_samples=10000, n_estimators=50, num_leaves=25; total time=   1.6s
[CV] END boosting_type=goss, learning_rate=0.1, min_child_samples=10000, n_estimators=50, num_leaves=25; total time=   1.2s
[CV] END boosting_type=gbdt, learning_rate=0.01, min_child_samples=100, n_estimators=50, num_leaves=5; total time=   2.9s
[CV] END boosting_type=gbdt, learning_rate=0.01, min_child_samples=100, n_estimators=50, num_leaves=5; total time=   2.9s
[CV] END boosting_type=gbdt, learning_rate=0.001, min_child_samples=50, n_estimators=500, num_leaves=10; total time=  18.4s
[CV] END boosting_type=gbdt, learning_rate=0.001, min_child_samples=50, n_estimators=500, num_leaves=10; total time=  17.5s


...

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /usr/local/src/LightGBM/src/boosting/rf.hpp, line 35 .



[CV] END boosting_type=rf, learning_rate=0.005, min_child_samples=1000, n_estimators=200, num_leaves=500; total time=   2.1s


[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /usr/local/src/LightGBM/src/boosting/rf.hpp, line 35 .



[CV] END boosting_type=rf, learning_rate=0.005, min_child_samples=1000, n_estimators=200, num_leaves=500; total time=   2.5s
[CV] END boosting_type=dart, learning_rate=0.2, min_child_samples=20, n_estimators=50, num_leaves=5; total time=   3.0s
[CV] END boosting_type=dart, learning_rate=0.2, min_child_samples=20, n_estimators=50, num_leaves=5; total time=   3.3s


RandomizedSearchCV(cv=2, estimator=LGBMClassifier(device='gpu'),
                   param_distributions={'boosting_type': ['gbdt', 'dart',
                                                          'goss', 'rf'],
                                        'learning_rate': [0.1, 0.01, 0.05,
                                                          0.001, 0.005, 0.2,
                                                          0.3],
                                        'min_child_samples': [20, 50, 100, 500,
                                                              1000, 5000,
                                                              10000],
                                        'n_estimators': [50, 100, 200, 500,
                                                         1000],
                                        'num_leaves': [5, 10, 25, 50, 100,
                                                       500]},
                   scoring=<function log_loss at 0x7f4b6b3b1680>, verbo

In [15]:
df = pd.DataFrame(clf.predict_proba(test))

In [16]:
df

,0,1
0,0.448804,0.551196
1,0.327114,0.672886
2,0.712090,0.287910
3,0.570130,0.429870
4,0.699148,0.300852
...,...,...
19995,0.712090,0.287910
19996,0.642096,0.357904
19997,0.498718,0.501282
19998,0.529535,0.470465


In [25]:
df1 = pd.read_csv("../input/tabular-playground-series-nov-2022/sample_submission.csv",index_col="id")
df1["pred"]=df[1].values
df1.to_csv("subNN6-3.csv")